In [2]:
import os
import pandas as pd
import numpy as np
import pandas_datareader
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import yfinance as yf

def stockData(sList, stDate, endDate):
    """Take list of stocks and pull data for that stock and create technical indicator variables
    sList requires list of stocks, dates should be entered in format 'YYYY-MM-DD' """
   
    data = []
    for element in sList:
        try:
            temp = pdr.get_data_yahoo(element, start = stDate, end = endDate)
            #add an indentifier
            temp['Symbol'] = element
            temp['Close'] = temp['Close']
        except: 
            #If stock symbol cannot be found make it $1
            print (element)
            temp['Symbol'] = element
            temp['Close'] = 1
        else:
        #add data to list 
            data.append(temp)
        #convert to dataframe    
        df = pd.concat(data)
        #drop rows for which we will not have all data points calculated
        df = df.dropna()
        #dates above what user entered
        df = df.loc[stDate : endDate]
        #order and limit columns to those needed
        df = df.loc[:, ['Symbol', 'Close']]
        df['Day'] = pd.to_datetime(df.index)

    
    return(df)


In [2]:
#set working directory 
os.chdir("/Users/bkrei/Desktop/Bk's Stuff Desktop/School/Github NEW PATH/Portfolio-Challenge")

In [3]:
#Read in individual portfolios
portfolios = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/Portfolios.csv")

In [14]:
#List of all ticker symbols
symbols = portfolios['Symbol'].drop_duplicates()
symbols.head(300)

0       AAPL
1      AGTHX
2       AMZN
3        CHE
4       DOCU
5      GSGRX
6       ICLN
7      OEGAX
8       PBFS
9        QQQ
10       SPY
13     BRK-B
14     CMCSA
15       CVS
16       DIS
17        GE
18       JNJ
19       JPM
20      NFLX
22      SBUX
23       XOM
28        BA
29         C
30      CHWY
31      CVRS
32      DELL
33       IAG
37       BAM
38       BEP
39       CGC
       ...  
382       KR
383     UUUU
384     ZIXI
385     Lulu
390      MTZ
393     goog
394      Dis
397      NOC
399      TPL
401     ANET
402     INGN
403      MED
404     SIVB
405     ALXN
407      RTN
409     ACST
411      DNR
413       NM
423       JP
426        X
441     CELG
442     COUP
443     CTSH
449     FTNT
450      HON
473      SAM
477      CXO
478     GWPH
480     MOMO
482     RVNC
Name: Symbol, Length: 238, dtype: object

In [9]:
symbols.to_csv("symbols.csv", header = False)

In [10]:
#Pass the ticker list to the function, print invalid stock symbols and create dataframe with intial prices
beg_prices = stockData(list(symbols), '2019-09-06', '2019-09-06')
beg_prices.shape

CASHPIO


(237, 3)

In [12]:
#Pull SNAP which for some reason is the only stock with an issue.
er = stockData(['SNAP'], '2019-09-06', '2019-09-06')

In [13]:
#Combine the main and SNAP dataframes
frames = [beg_prices, er]
beg_prices = pd.concat(frames)

In [15]:
beg_prices.to_csv("beg_prices.csv")

In [4]:
beg_prices = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/beg_prices.csv")
combined = pd.merge(portfolios, beg_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [5]:
combined.head(5)

,Name,Symbol,% of Portfolio,Cost Basis,Date,Close,Day
0,Adrienne Villano,AAPL,0.07,14000,2019-09-06,213.259995,2019-09-06
1,Adrienne Villano,AGTHX,0.11,22000,2019-09-06,50.000000,2019-09-06
2,Adrienne Villano,AMZN,0.07,14000,2019-09-06,1833.510010,2019-09-06
3,Adrienne Villano,CHE,0.18,36000,2019-09-06,439.820007,2019-09-06
4,Adrienne Villano,DOCU,0.05,10000,2019-09-06,56.270000,2019-09-06


In [6]:
#Calc # of shares purchased
combined["# of Shares"] = combined["Cost Basis"]/ combined["Close"]
#Convert names to proper case
#combined['Name'] = list(map(lambda x: x.title(), combined['Name']))
#combined.head()

In [20]:
combined.to_csv("initial portfolios.csv")

# Simple Comparison (Initial vs Current Value) START HERE TO UPDATE

In [15]:
#Select date to compare to initial prices
cur_prices = stockData(list(symbols), '2019-09-19', '2019-09-19')

TSS
CASHPIO


In [36]:
#Pull SNAP which for some reason is the only stock with an issue.
er2 =  stockData(['SNAP'], '2019-09-19', '2019-09-19')

TSS


In [37]:
#Combine the main and SNAP dataframes
frames2 = [cur_prices, er2]
cur_prices = pd.concat(frames2)

In [38]:
cur_prices.sort_values("Symbol", inplace = True)
cur_prices.drop_duplicates(subset = "Symbol", inplace = True)

In [39]:
cur_prices.to_csv("cur_prices.csv")

In [12]:
cur_prices= pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/cur_prices.csv")
cur_prices.head()


,Date,Symbol,Close,Day
0,2019-09-12,AAL,29.940001,2019-09-12
1,2019-09-12,AAPL,223.089996,2019-09-12
2,2019-09-12,ACB,5.890000,2019-09-12
3,2019-09-12,ACER,3.160000,2019-09-12
4,2019-09-12,ACST,2.150000,2019-09-12


In [41]:
detail = pd.merge(combined, cur_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [42]:
detail.head()

,Name,Symbol,% of Portfolio,Cost Basis,Close_x,Day_x,# of Shares,Close_y,Day_y
0,Adrienne Villano,AAPL,0.07,14000,213.259995,2019-09-06,65.647568,220.960007,2019-09-19
1,Adrienne Villano,AGTHX,0.11,22000,50.000000,2019-09-06,440.000000,50.560001,2019-09-19
2,Adrienne Villano,AMZN,0.07,14000,1833.510010,2019-09-06,7.635628,1821.500000,2019-09-19
3,Adrienne Villano,CHE,0.18,36000,439.820007,2019-09-06,81.851665,425.260010,2019-09-19
4,Adrienne Villano,DOCU,0.05,10000,56.270000,2019-09-06,177.714589,63.360001,2019-09-19


In [43]:
detail['Current_Value'] = detail['Close_y'] * detail['# of Shares']
detail['Perc_Return'] = detail['Close_y']/detail['Close_x']  -1
detail.head(5)

,Name,Symbol,% of Portfolio,Cost Basis,Close_x,Day_x,# of Shares,Close_y,Day_y,Current_Value,Perc_Return
0,Adrienne Villano,AAPL,0.07,14000,213.259995,2019-09-06,65.647568,220.960007,2019-09-19,14505.487075,0.036106
1,Adrienne Villano,AGTHX,0.11,22000,50.000000,2019-09-06,440.000000,50.560001,2019-09-19,22246.400604,0.011200
2,Adrienne Villano,AMZN,0.07,14000,1833.510010,2019-09-06,7.635628,1821.500000,2019-09-19,13908.296036,-0.006550
3,Adrienne Villano,CHE,0.18,36000,439.820007,2019-09-06,81.851665,425.260010,2019-09-19,34808.239954,-0.033104
4,Adrienne Villano,DOCU,0.05,10000,56.270000,2019-09-06,177.714589,63.360001,2019-09-19,11259.996463,0.126000


In [44]:
detail.to_csv("detail.csv")

### Individual Performance

In [45]:
ranking = detail.groupby("Name")['Cost Basis', 'Current_Value'].sum().reset_index().sort_values("Current_Value", ascending=False)

In [46]:
ranking['Perc_Return'] = ranking['Current_Value']/ranking['Cost Basis'] -1
ranking['Rank'] = ranking['Perc_Return'].rank(ascending=False)
ranking.head(10)

,Name,Cost Basis,Current_Value,Perc_Return,Rank
12,Colton Young,200000,227517.278304,0.137586,1.0
23,JOELEAN MCBRIDE,200000,220323.620583,0.101618,2.0
47,Robert Kovacs,200000,218764.317391,0.093822,3.0
45,Randy Benedict,200000,212029.947785,0.060150,4.0
54,Trudy Seeber,200000,210778.845822,0.053894,5.0
42,Pete Lindner,200000,210389.476981,0.051947,6.0
27,Joseph Lindner,200000,210154.001817,0.050770,7.0
19,Elise Shaver,200000,209420.857026,0.047104,8.0
37,Marie Barkamian,200000,208405.634540,0.042028,9.0
40,Nick Capullo,200000,206805.830638,0.034029,10.0


In [47]:
ranking.to_csv("individual rankings.csv")

### Best Individual Stock Returns

In [48]:
best_stocks = detail.drop_duplicates(['Symbol']).reset_index().sort_values(['Perc_Return'], ascending=False)

In [49]:
best_stocks = best_stocks.loc[:, ['Symbol', 'Perc_Return']]
best_stocks.head(10)

,Symbol,Perc_Return
79,PIR,0.579545
237,RVNC,0.294388
171,HOME,0.269337
97,PLUG,0.243363
165,HOV,0.235619
207,ACER,0.234875
125,MARK,0.224490
203,FNMAL,0.215467
224,DNR,0.168142
120,RUN,0.161677


In [50]:
best_stocks.to_csv("stock ranking.csv")

# Trend

In [ ]:
#Pull Only necessary columns
combined = combined.loc[:, ['Name', 'Symbol', '# of Shares']]

In [ ]:
Price_History = stockData(stocks['Ticker'], '2019-08-01', '2019-09-06')

In [ ]:
historical = pd.merge(combined, Price_History, how = 'outer', on = "Symbol", validate="many_to_many")


In [ ]:
historical['Value'] = historical['# of Shares'] * historical['Close']
historical.head()

In [ ]:
trend_data = historical.groupby(['Name', 'Day'])['Value'].sum().reset_index().sort_values(['Name', 'Day'])

In [ ]:
trend_data.head()

In [1]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode